In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
!unzip -uq gdrive/MyDrive/cs492i_project/Vin_CXR_512.zip -d Vin_CXR_512/
!unzip -uq gdrive/MyDrive/cs492i_project/train.csv.zip train.csv

In [ ]:
! pip install tensorboardX tensorboard pyyaml webcolors 
! pip install -U git+https://github.com/albumentations-team/albumentations

     |████████████████████████████████| 124 kB 9.8 MB/s 
  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-1aa4ts0l
  Running command git clone -q https://github.com/albumentations-team/albumentations /tmp/pip-req-build-1aa4ts0l
     |████████████████████████████████| 47.6 MB 1.7 MB/s 
  Created wheel for albumentations: filename=albumentations-1.1.0-py3-none-any.whl size=105152 sha256=73a80e6b7b06af556b07ad628e7cd514922f67f6e6eb1672f6397f851312add6
  Stored in directory: /tmp/pip-ephem-wheel-cache-j271lkzz/wheels/3a/25/ed/ec3b518e7a332d7f0a3bb37c280e1b784cf2f79b94b3c7d00b
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
!pip install torchvision

In [ ]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path 
import os
import matplotlib.pyplot as plt
import json 
import cv2 
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2
import torchvision 
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm

In [ ]:
traindata= pd.read_csv('train.csv')
traindata.head(10)

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN
5,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0
6,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0
7,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0
8,d3637a1935a905b3c326af31389cb846,Aortic enlargement,0,R10,1329.0,743.0,1521.0,958.0
9,afb6230703512afc370f236e8fe98806,Pulmonary fibrosis,13,R9,1857.0,1607.0,2126.0,2036.0


In [ ]:
obj_list = ["Aortic_enlargement",
            "Atelectasis",
            "Calcification",
            "Cardiomegaly",
            "Consolidation",
            "ILD",
            "Infiltration",
            "Lung_Opacity",
            "Nodule/Mass",
            "Other_lesion",
            "Pleural_effusion",
            "Pleural_thickening",
            "Pneumothorax",
            "Pulmonary_fibrosis"]
len(obj_list)

14

In [ ]:
pic_id = os.listdir('./Vin_CXR_512/Vin_CXR_512/train')[0][:-4]
smalldata = traindata[traindata['image_id'] == pic_id]
lol = set(smalldata['class_id'])
result = np.zeros(14)
for i in lol:
  result[i] = 1
lol.discard(14)
lol

{1, 6, 11, 13}

In [ ]:
data_transform= {
        'train': Compose([
            Resize(512,512),
            HorizontalFlip(),
            RandomResizedCrop(512,512,p=0.5),
            ShiftScaleRotate(rotate_limit=10),  
            RandomBrightnessContrast(),
            Normalize(max_pixel_value=1),
            ToTensorV2()
        ]),
        'val': Compose([
            Resize(512,512),
            Normalize(max_pixel_value=1),
            ToTensorV2()
        ]),
    }

In [ ]:
class CXR_DataLoader(torch.utils.data.Dataset):
  def __init__(self, mode, transform=data_transform):
    super(CXR_DataLoader, self).__init__()
    self.mode = mode
    self.transform = transform[self.mode]

  def __len__(self):
    return len(os.listdir('/content/Vin_CXR_512/Vin_CXR_512/' + self.mode))

  def __getitem__(self, index):

    indexlist = os.listdir('/content/Vin_CXR_512/Vin_CXR_512/' + self.mode)
    pic_id = indexlist[index][:-4]               # Removing 4 characters relating to .png

    X = plt.imread('/content/Vin_CXR_512/Vin_CXR_512/' + self.mode + '/' + indexlist[index])
    
    #dev Hoang
    #print(X.max())

    X = self.transform(image=X)['image']

    smalldata = traindata[traindata['image_id'] == pic_id]
    lol = set(smalldata['class_id'])
    result = np.zeros(14)

    for i in lol:
      if i != 14:
        result[i] = 1
    y = (pic_id, result)

    return X, y

In [ ]:
train_dataloader = torch.utils.data.DataLoader(CXR_DataLoader(mode='train'), batch_size=16,shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(CXR_DataLoader(mode='val'), batch_size=1,shuffle=False, num_workers=2)

In [ ]:
! git clone https://github.com/levi3001/Yet-Another-EfficientDet-Pytorch.git


Cloning into 'Yet-Another-EfficientDet-Pytorch'...
remote: Enumerating objects: 844, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 844 (delta 45), reused 70 (delta 41), pack-reused 765
Receiving objects: 100% (844/844), 121.22 MiB | 32.56 MiB/s, done.
Resolving deltas: 100% (482/482), done.


In [ ]:
cd Yet-Another-EfficientDet-Pytorch/

/content/Yet-Another-EfficientDet-Pytorch


In [ ]:
# resnet 50

model=torchvision.models.resnet50(pretrained=True)
model.fc= nn.Linear(2048,14)
model

In [ ]:
#efficientnet b0

from efficientnet import EfficientNet
from backbone import EfficientDetBackbone
model = EfficientNet.from_pretrained('efficientnet-b3', load_weights=False, num_classes=14)
model.parameters
#model = EfficientDetBackbone( )

<bound method Module.parameters of EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    (conv): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        (conv): Conv2d(40, 40, kernel_size=(3, 3), stride=(1,), groups=40, bias=False)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        (conv): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
      )
      (_se_expand): Conv2dStaticSamePadding(
        (conv): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
      )
      (_project_conv): Conv2dStaticSamePadding(
        (conv): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (_bn2): BatchNorm2d(24, eps=0.001, momentum=0

In [ ]:
# model_backbone=model.backbone_net
# model_backbone

In [ ]:
device='cuda'
def train_model(model, epoches, optimizer):
  # model = Model()

  loss_function = nn.BCEWithLogitsLoss()
  model.to(device)

  bestf1 = 0
  for epoch in range(epoches):
    # Train Models
    model.train()
    loss0 = 0
    train_number_data = 0
    for batch_id, (X, y) in tqdm(enumerate(train_dataloader)):
      pic_id, labels = y
      X = X.to(device=device)
      labels = labels.to(device=device)

      #Feed Forward
      output = model(X)

      #Loss Calculate
      loss = loss_function(output, labels)
      loss0 += loss.item()
      #Back propagate
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_number_data += X.shape[0]
    print('Epoch', epoch, ': Train Loss', loss0/train_number_data)
    
    # Evaluate models

    model.eval()
    with torch.no_grad():
      test_loss = 0
      test_accuracy = 0
      test_num_data = 0
      tp = torch.zeros(14).to(device)
      fp = torch.zeros(14).to(device)
      fn = torch.zeros(14).to(device)

      for batch_idx, (X, y) in tqdm(enumerate(test_dataloader)):
        pic_id, labels = y
        X = X.to(device=device)
        labels = labels.to(device=device)
        output = model(X)

        loss = loss_function(output, labels)
        test_loss += loss.item()*X.shape[0]

        # Calculate Accuracy
        output = torch.sigmoid(output)
        lol2 = torch.zeros(size=output.shape).to(device)
        lol2[output > 0.5] = 1
        accuracy = torch.sum(lol2 == labels, dim=0)/(X.shape[0])
        test_accuracy += accuracy*X.shape[0]

        # Calculate Mean F1

        tp += torch.sum((lol2 == labels) & (labels == 1), dim=0)
        fp += torch.sum((lol2 != labels) & (labels == 1), dim=0)
        fn += torch.sum((lol2 != labels) & (labels == 0), dim=0)

        test_num_data +=X.shape[0]
      
      tp /= test_num_data
      fp /= test_num_data
      fn /= test_num_data
      test_loss /= test_num_data
      test_accuracy /= test_num_data
      f1 = tp/(tp + 0.5*(fn+fp))
      mean_f1 = torch.mean(f1)

      if mean_f1.item() > bestf1:
        print('Save best:')
        PATH = 'b3_efficient_best_2.pth'
        torch.save(model.state_dict(), PATH)
        bestf1 = mean_f1.item()
        
    print('Test Accuracy', torch.mean(test_accuracy).item(), '. Test Loss', loss.item(), '. F1:', mean_f1.item())
    #print(loss)
    #print(test_accuracy)

  print(mean_f1)

In [ ]:
# import torchvision
# model = torchvision.models.resnet50(pretrained= True, progress= True)
# model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# # Here the size of each output sample is set to 2.
# # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model_ft.fc = nn.Linear(num_ftrs, 2)

# model_ft = model_ft.to(device)

# criterion = nn.CrossEntropyLoss()

# # Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
#train efficienet b0

optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-4, last_epoch=- 1, verbose=False)
train_model(model, 100, optimizer)

0it [00:00, ?it/s]

Epoch 0 : Train Loss 0.03167624215041939


0it [00:00, ?it/s]

Save best:
Test Accuracy 0.7879083752632141 . Test Loss 0.46154833904334475 . F1: 0.10905508697032928


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

Test Accuracy 0.8337395787239075 . Test Loss 0.3442843434874833 . F1: 0.2791060507297516


0it [00:00, ?it/s]

Epoch 25 : Train Loss 0.025379374911306273


0it [00:00, ?it/s]

In [ ]:
#train resnet

optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-4, last_epoch=- 1, verbose=False)
train_model(model, 100, optimizer)

In [ ]:
f1

NameError: ignored

In [ ]:
PATH = '/content/gdrive/MyDrive/cs492i_project/efficientnet.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
!cp b3_efficient_best_2.pth.pth /content/gdrive/MyDrive/cs492i_project/efficientnet_b3_best_2.pth 

In [ ]:
from efficientnet_pytorch import EfficientNet
efficientnet = torch.load('./gdrive/MyDrive/cs492i_project/efficientnet')

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=14)
model.load_state_dict(efficientnet)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [ ]:
model=torchvision.models.resnet50(pretrained=True)
model.fc= nn.Linear(2048,14)
model.load_state_dict(torch.load( 'resnet50_best.pth'))


<All keys matched successfully>

In [ ]:
model = model.cuda()
model.eval()
loss_function = nn.BCEWithLogitsLoss()
with torch.no_grad():
  test_loss = 0
  test_accuracy = 0
  test_num_data = 0
  tp = torch.zeros(14).to(device)
  fp = torch.zeros(14).to(device)
  fn = torch.zeros(14).to(device)



  for batch_idx, (X, y) in tqdm(enumerate(test_dataloader)):
    pic_id, labels = y
    X = X.to(device=device)
    labels = labels.to(device=device)
    output = model(X)

    loss = loss_function(output, labels)
    test_loss += loss.item()*X.shape[0]

    # Calculate Accuracy
    output = torch.sigmoid(output)
    lol2 = torch.zeros(size=output.shape).to(device)
    lol2[output > 0.5] = 1
    accuracy = torch.sum(lol2 == labels, dim=0)/(X.shape[0])
    test_accuracy += accuracy*X.shape[0]

    # Calculate Mean F1

    tp += torch.sum((lol2 == labels) & (labels == 1), dim=0)
    fp += torch.sum((lol2 != labels) & (labels == 1), dim=0)
    fn += torch.sum((lol2 != labels) & (labels == 0), dim=0)

    test_num_data +=X.shape[0]

  tp /= test_num_data
  fp /= test_num_data
  fn /= test_num_data
  test_loss /= test_num_data
  test_accuracy /= test_num_data
  f1 = tp/(tp + 0.5*(fn+fp))
  mean_f1 = torch.mean(f1)

0it [00:00, ?it/s]

In [ ]:
torch.sigmoid(torch.Tensor([[1, 5, 6], [2, 3, 1]]))

tensor([[0.7311, 0.9933, 0.9975],
        [0.8808, 0.9526, 0.7311]])

In [ ]:
mean_f1

tensor(0.6062, device='cuda:0')

In [ ]:
f1

tensor([0.8954, 0.3956, 0.4319, 0.8745, 0.5641, 0.5442, 0.4577, 0.6180, 0.5788,
        0.4899, 0.7855, 0.6180, 0.5333, 0.7001], device='cuda:0')

In [ ]:
test_accuracy

tensor([0.8532, 0.9374, 0.8623, 0.8635, 0.9420, 0.9238, 0.8760, 0.7440, 0.8510,
        0.6257, 0.9056, 0.7201, 0.9841, 0.7691], device='cuda:0')